<a href="https://colab.research.google.com/github/marcelogbrito/machine-learning-negocios/blob/main/DeteccaoFraudes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from scipy import special

import itertools
from datetime import timedelta
from IPython.display import display

In [ ]:
fraudes = pd.read_csv('Fraud_Data.csv', header=0)

FileNotFoundError: ignored

In [ ]:
ips = pd.read_csv("IpAddress_to_Country.csv",header=0)

In [ ]:
fraudes.dtypes

In [ ]:
for col in fraudes.columns:
    print("# de valores nulos em {}: {}".format(col, fraudes[col].isnull().sum()))

In [ ]:
fraudes.describe()

In [ ]:
for col in ips.columns:
    print("{}: {}".format(col, ips[col].dtype))

In [ ]:
# checar se os numeros de ip sao inteiros ou nao com lambdas
ip_floats = ips.limite_inferior_ip.apply(lambda x: not x.is_integer()).sum()
print("Número de limites inferiores de endereço IP com decimal não-zero: {}".format(ip_floats))

In [ ]:
ips.limite_inferior_ip = ips.limite_inferior_ip.astype(int)

In [ ]:
for col in ips.columns:
    print("# de valores nulos em {}: {}".format(col, ips[col].isnull().sum()))

In [ ]:
#checar se há outliers
display(ips[ips.limite_inferior_ip == ips.limite_inferior_ip.max()])
display(ips[ips.limite_superior_ip == ips.limite_superior_ip.max()])

In [ ]:
ips.describe()

In [ ]:
fraudes.describe()

In [ ]:
def isp_counts(isp):
    pais = ips[(ips.limite_inferior_ip <= isp) & (ips.limite_superior_ip >= isp)]
    if pais.shape[0] == 1:
        return print("Único")
    elif pais.shape[0] > 1:
        return print("Vários países")
    elif pais.shape[0] == 0:
        if isp > ips.limite_superior_ip.max():
            return print("Muito grande")
        elif isp < ips.limite_inferior_ip.min():
            return print("Muito pequeno")
        else:
            return print("Faltando")
        
ip_counts = fraudes.ip.apply(isp_counts)
ip_counts.value_counts()

In [ ]:
def ip_para_pais(ip):
    tmp = ips[(ips.limite_inferior_ip <= ip) & (ips.limite_superior_ip >= ip)].pais
    if tmp.shape[0] == 1:
        return tmp.iloc[0]
    elif tmp.shape[0] < 1:
        return print("Faltando")
    elif tmp.shape[0] > 1:
        return print("Erro")

fraudes["pais"] = fraudes.ip.apply(ip_para_pais)

In [ ]:
fraudes.head()

## Analise exploratoria

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

### Genero x fraude

In [ ]:
sns.countplot(x="Gênero", hue="fraude", data=fraudes)
plt.xlabel("Gênero", fontsize=15)
plt.ylabel("Casos", fontsize=15)

#### Hue indica de acordo com a variavel o que é fraude e o que não é

### Fraude x valor

In [ ]:
sns.factorplot(x="fraude", y="valor", data=fraudes)
plt.xlabel("Não são fraude (0) / Fraude (1)", fontsize=15)
plt.ylabel("Valor de compra", fontsize=15)

### Fraude x idade

In [ ]:
sns.factorplot(x="fraude", y="idade", data=fraudes)
plt.xlabel("Não são fraude (0) / "Fraude (1)", fontsize=15)
plt.ylabel("Idade", fontsize-15)

### Fraude x fonte

In [ ]:
sns.countplot(x="fonte", hue="fraude", data=fraudes)
plt.xlabel("Fonte de marketing", fontsize=15)
plt.ylabel("Contagem", fontsize=15)

## Explorando os países

In [ ]:
sums_f = fraudes[fraudes["fraude"] == 1]["fraude"].groupby(fraudes.pais).sum()
sums_f = sums_f.to_frame().reset_index()

sums_nf = fraudes[fraudes["fraude"] == 0]["fraude"].groupby(fraudes.pais).sum()
sums_nf = sums_nf.to_frame().reset_index()

In [ ]:
f, ax = plt.subplots(figsize=(20,25))
sns.set_color_codes("muted")
sns.barplot(x="fraude", y="pais", data=sums_f,
            label="Total", color="b")
plt.xlabel("Contagem",fontsize=15)
plt.ylabel("Países",fontsize=15)

## Explorando os IDs e IPs

In [ ]:
tmp_df = pd.DataFrame(columns=("fraude","id_por_dispositivo","id_por_ip"))
tmp_df["fraude"] = fraudes["fraude"]

userid_count = fraudes.groupby["id_dispositivo"].id.count()
tmp_df[id_por_dispositivo"] = fraude_df.id_dispositivo.apply(lambda x: userid_count[x])

userid_count2 = fraudes.groupby["ip"].id.count()
tmp_df["id_por_ip"] = fraudes.ip.apply(lambda x: userid_count2[x])